# CrewAI + LlamaIndex Cookbook

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/cookbooks/crewai_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This cookbook shows you how to build an advanced research assistant by plugging LlamaIndex-powered tools into a CrewAI-powered multi-agent setup.

LlamaIndex is a framework enabling developers to easily build LLM-powered applications over their data; it contains production modules for indexing, retrieval, and prompt/agent orchestration. A core use case is building a generalized QA interface enabling knowledge synthesis over complex questions.

Plugging a LlamaIndex RAG pipeline as a tool into a CrewAI agent setup enables even more sophisticated/advanced research flows as shown below. We show the following:

1. **Setup a Simple Calculator Agent**: We port over the set of tools available on LlamaHub (https://llamahub.ai/).
2. **Setup a Financial Analyst Agent**: We plug in a RAG query engine as a tool accessible to a CrewAI agent.

In [33]:
!pip install crewai
!pip install crewai-tools

In [34]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool

In [35]:
!pip install llama-index-core #rag
!pip install llama-index-readers-file
!pip install llama-index-tools-wolfram-alpha
!pip install 'crewai[tools]'

## Setup a Simple Calculator Agent

In this section we setup a crew of agents that can perform math and generate a 10-question multiple choice test (with answers) from it.

#### Wolfram Alpha Tool
Let's setup Wolfram Alpha as a general math computation tool.

In [36]:
from llama_index.tools.wolfram_alpha import WolframAlphaToolSpec

wolfram_spec = WolframAlphaToolSpec(app_id="<app_id>")
wolfram_tools = wolfram_spec.to_tool_list()

In [37]:
wolfram_tools[0]("(7 * 12 ^ 10) / 321")

ToolOutput(content='Error 1: Invalid appid', tool_name='wolfram_alpha_query', raw_input={'args': ('(7 * 12 ^ 10) / 321',), 'kwargs': {}}, raw_output='Error 1: Invalid appid', is_error=False)

In [38]:
wolfram_tools[0]("How many calories are there in a pound of apples")

ToolOutput(content='Error 1: Invalid appid', tool_name='wolfram_alpha_query', raw_input={'args': ('How many calories are there in a pound of apples',), 'kwargs': {}}, raw_output='Error 1: Invalid appid', is_error=False)

In [39]:
crewai_wolfram_tools = [LlamaIndexTool.from_tool(t) for t in wolfram_tools]

In [20]:
print(crewai_wolfram_tools[0].description)

Tool Name: wolfram_alpha_query
Tool Arguments: {'query': {'description': None, 'type': 'str'}}
Tool Description: wolfram_alpha_query(query: str)

        Make a query to wolfram alpha about a mathematical or scientific problem.

        Example inputs:
            "(7 * 12 ^ 10) / 321"
            "How many calories are there in a pound of strawberries"

        Args:
            query (str): The query to be passed to wolfram alpha.

        


In [40]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [41]:
# Define your agents with roles and goals
calculator_agent = Agent(
    role="Calculator",
    goal="Solve complex math problems",
    backstory="""You are an AI computer that has access to Wolfram Alpha to perform complex computations.""",
    verbose=True,
    tools=crewai_wolfram_tools, # Make sure this list contains your actual Wolfram tool
)
teacher_agent = Agent(
    role="Math Teacher",
    goal="Make tests for students.",
    backstory="""You are a math teacher preparing a simple arithmetic test for your 2nd grade students.""",
    verbose=True,
    allow_delegation=False,
)

# Create tasks for your agents
task1 = Task(
    description="""Using the math operators (+, -, *, /), and numbers from 1-100, generate 10 medium-difficulty arithmetic problems
  that consist of numbers/operators/parentheses in different ways.

  Generate the actual answer for each problem too. Use the Wolfram tool for this.
  """,
    expected_output="10 arithmetic expressions with the actual answers",
    agent=calculator_agent,
)
# Generate a multiple choice math test with answers. Will later give this test out to students (without answers of course).

task2 = Task(
    description="""Using the generated expressions/answers, generate a multiple choice for students.
  Each question should have 4 options, one being the correct answer. """,
    expected_output="Test with 10 multiple choice questions",
    agent=teacher_agent,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[calculator_agent, teacher_agent],
    tasks=[task1, task2],
    verbose=True,
)

# Now you would typically run the crew:
# result = crew.kickoff()
# print(result)

In [42]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cd298652-8fb8-4019-8f02-d740859333c3                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: bb9b1c53-a175-42cc-8261-aeb70c652c96
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: bb9b1c53-a175-42cc-8261-aeb70c652c96
       Status: Executing Task...
    └── 🤖 Agent: Calculator
            Status: In Progress

# Agent: Calculator
## Task: Using the math operators (+, -, *, /), and numbers from 1-100, generate 10 medium-difficulty arithmetic problems
  that consist of numbers/operators/parentheses in different ways.

  Generate the actual answer for each problem too. Use the Wolfram tool for this.
  


# Agent: Calculator
## Final Answer: 
1. (35 + 47) * 2 = 164, 2. 100 - (25 / 5) + 36 = 111, 3. (12 * 7) - (4 * 3) = 84, 4. 64 / 8 + (15 - 5) = 24, 5. 90 - (45 * (2 - 1)) = 45, 6. (8 * 6 - 4) / 2 + 5 = 29, 7. (50 / 2) + (30 - 10) = 45, 8. (25 + 5) * (3 - 1) = 60, 9. 60 / (3 + 1) + 20 = 25, 10. (4 + 6) * 10 - 5 = 95




🚀 Crew: crew
└── 📋 Task: bb9b1c53-a175-42cc-8261-aeb70c652c96
       Status: Executing Task...
    └── 🤖 Agent: Calculator
            Status: ✅ Completed

...

🚀 Crew: crew
├── 📋 Task: bb9b1c53-a175-42cc-8261-aeb70c652c96
│      Assigned to: Calculator
│      Status: ✅ Completed
│   └── 🤖 Agent: Calculator
│           Status: ✅ Completed
└── 📋 Task: 5f5b5109-ee26-4645-b510-aa624dbe5656
       Status: Executing Task...
    └── 🤖 Agent: Math Teacher
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: bb9b1c53-a175-42cc-8261-aeb70c652c96
│      Assigned to: Calculator
│      Status: ✅ Completed
│   └── 🤖 Agent: Calculator
│           Status: ✅ Completed
└── 📋 Task: 5f5b5109-ee26-4645-b510-aa624dbe5656
       Assigned to: Math Teacher
       Status: ✅ Completed
    └── 🤖 Agent: Math Teacher
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5f5b5109-ee26-4645-b510-aa624dbe5656                                                                     │
│  Agent: Math Teacher                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cd298652-8fb8-4019-8f02-d740859333c3                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

######################
**Arithmetic Test: Multiple Choice Questions**

1. What is (35 + 47) * 2?  
   A) 150  
   B) 164  
   C) 172  
   D) 120  

2. What is 100 - (25 / 5) + 36?  
   A) 100  
   B) 95  
   C) 111  
   D) 110  

3. What is (12 * 7) - (4 * 3)?  
   A) 66  
   B) 60  
   C) 84  
   D) 70  

4. What is 64 / 8 + (15 - 5)?  
   A) 20  
   B) 28  
   C) 24  
   D) 25  

5. What is 90 - (45 * (2 - 1))?  
   A) 50  
   B) 40  
   C) 55  
   D) 45  

6. What is (8 * 6 - 4) / 2 + 5?  
   A) 24  
   B) 25  
   C) 29  
   D) 28  

7. What is (50 / 2) + (30 - 10)?  
   A) 40  
   B) 45  
   C) 50  
   D) 60  

8. What is (25 + 5) * (3 - 1)?  
   A) 50  
   B) 60  
   C) 70  
   D) 80  

9. What is 60 / (3 + 1) + 20?  
   A) 23  
   B) 30  
   C) 25  
   D) 20  

10. What is (4 + 6) * 10 - 5?  
    A) 95  
    B) 85  
    C) 75  
    D) 90  

**Answers Key for Teachers:**
1. B) 164  
2. C) 111  
3. C) 84  
4. C) 24  
5. D) 45  
6. C) 29  
7. B) 45  
8. B) 60  
9. C) 25  
10. A) 95


## Setup a Simple Financial Analyst Agent

In this section we setup a crew that uses a LlamaIndex RAG pipeline over an Uber 10K as its core query tool.

In [24]:
!wget "https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf" -O uber_10k.pdf

--2025-04-19 12:26:27--  https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf
Resolving s23.q4cdn.com (s23.q4cdn.com)... 68.70.205.4, 68.70.205.3, 68.70.205.1, ...
Connecting to s23.q4cdn.com (s23.q4cdn.com)|68.70.205.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829436 (2.7M) [application/pdf]
Saving to: ‘uber_10k.pdf’

uber_10k.pdf        100%[===================>]   2.70M  4.35MB/s    in 0.6s    

2025-04-19 12:26:28 (4.35 MB/s) - ‘uber_10k.pdf’ saved [2829436/2829436]



In [25]:
!pip install -U llama-index llama-index-llms-openai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 6.7 MB/s eta 0:00:00


In [26]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI


reader = SimpleDirectoryReader(input_files=["uber_10k.pdf"])
docs = reader.load_data()

In [27]:
docs[1].get_content()

'69 \nCountries\n10K+ \nCities\n$65B \nGross Bookings\n111M \nMAPCs\n7B \nTrips\nA global tech \nplatform at \nmassive scale\nServing multiple multi-trillion \ndollar markets with products \nleveraging our core technology \nand infrastructure\nWe believe deeply in our bold mission. Every minute \nof every day, consumers and Drivers on our platform \ncan tap a button and get a ride or tap a button and \nget work. We revolutionized personal mobility with \nridesharing, and we are leveraging our platform to \nredefine the massive meal delivery and logistics \nindustries. The foundation of our platform is our \nmassive network, leading technology, operational \nexcellence, and product expertise. Together, these \nelements power movement from point A to point B.'

In [28]:
llm = OpenAI(model="gpt-4o")
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

/usr/local/lib/python3.11/dist-packages/llama_index/core/indices/base.py:110: DeprecationWarning: Call to deprecated method get_doc_id. ('get_doc_id' is deprecated, access the 'id_' property instead.) -- Deprecated since version 0.12.2.
  docstore.set_document_hash(doc.get_doc_id(), doc.hash)


In [29]:
# try out query engine tool

query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Uber 2019 10K Query Tool",
    description="Use this tool to lookup the 2019 Uber 10K Annual Report",
)

In [30]:
query_tool.args_schema.schema()

<ipython-input-30-fe53991c08af>:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  query_tool.args_schema.schema()


{'description': 'Schema for query tool.',
 'properties': {'query': {'description': 'Search query for the query tool.',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'QueryToolSchema',
 'type': 'object'}

## Generate a Research Report

Now that we have the query interface over the Uber 10K setup with LlamaIndex, we can now generate a research report with CrewAI.


We follow the agent/writer setup in the CrewAI quickstart tutorial, and modify it to use the query tool.

We then run it and analyze the results.

In [31]:
# Define your agents with roles and goals
researcher = Agent(
    role="Senior Financial Analyst",
    goal="Uncover insights about different tech companies",
    backstory="""You work at an asset management firm.
  Your goal is to understand tech stocks like Uber.""",
    verbose=True,
    allow_delegation=False,
    tools=[query_tool],
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=False,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of Uber's risk factors in 2019.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the headwinds that Uber faces.
  Your post should be informative yet accessible, catering to a casual audience.
  Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,  # You can set it to 1 or 2 to different logging levels
)

# # Instantiate your crew with a sequential process
# crew = Crew(
#     agents=[calculator_agent, teacher_agent],
#     tasks=[task1, task2],
#     verbose=True,
# )

In [32]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8e64bc45-1001-4537-9ebf-7633e3fc62c5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 363fc1d6-6b14-4c86-b9bc-3a88c932a489
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 363fc1d6-6b14-4c86-b9bc-3a88c932a489
       Status: Executing Task...
    └── 🤖 Agent: Senior Financial Analyst
            Status: In Progress

# Agent: Senior Financial Analyst
## Task: Conduct a comprehensive analysis of Uber's risk factors in 2019.


🤖 Agent: Senior Financial Analyst
    Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 363fc1d6-6b14-4c86-b9bc-3a88c932a489
│      Assigned to: Senior Financial Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Financial Analyst
│           Status: ✅ Completed
└── 📋 Task: e00f9bf7-07dc-45f2-9174-36800d586f23
       Status: Executing Task...
    └── 🤖 Agent: Tech Content Strategist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 363fc1d6-6b14-4c86-b9bc-3a88c932a489
│      Assigned to: Senior Financial Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Financial Analyst
│           Status: ✅ Completed
└── 📋 Task: e00f9bf7-07dc-45f2-9174-36800d586f23
       Assigned to: Tech Content Strategist
       Status: ✅ Completed
    └── 🤖 Agent: Tech Content Strategist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e00f9bf7-07dc-45f2-9174-36800d586f23                                                                     │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8e64bc45-1001-4537-9ebf-7633e3fc62c5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

######################
**Navigating the Storm: Uber’s Headwinds Ahead**

Uber has undoubtedly changed the way we think about transportation, but behind the glossy app and stylish rides, the road ahead is filled with some serious challenges. As a platform that connects millions of drivers and riders, Uber operates in a complex ecosystem where data security, regulatory compliance, and user retention play crucial roles. Each of these elements poses unique risks, threatening the very foundation of its business model. Let’s dive into some of the headwinds that Uber is facing today.

One of the first issues on Uber’s radar is the risk of data security breaches. With the acquisition of companies like Careem, the labyrinth of data only gets trickier. Any unintended slip-up could lead to major liabilities, which can be not just costly but also damaging to Uber’s reputation. When customer trust is on the line, the stakes are incredibly high. Imagine the public's reaction if sensitive user inform